# Import

In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import random

#torch
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

#Albumentation
import albumentations as A
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
classes=1

In [67]:
import sys
sys.path.append("../input/timm-pytorch-image-models/pytorch-image-models-master")
import timm

# Seed

In [68]:
SEED = 508

def random_seed(SEED):
    
    random.seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True

random_seed(SEED)

# CFG

In [69]:
class CFG:
    frac=1.0 #1.0#
    train=True#False#
    pretrained=True #False#
    model= "BERT"#"R34"#"E4"#
    train_bs=36
    valid_bs=18#12
    test_bs=36
    epochs=30
    fold=[0,1,2]
    classes=1
    seed= 719
    lr=2e-4
    es=4

# DataLoad

# Dataset

In [70]:
class SETIDataset(torch.utils.data.Dataset):
    def __init__(self,df,transform=None):
        self.df=df
        self.paths=df["path"].values
        self.labels=df["target"].values
        self.transform=transform
    def __len__(self):
        return len(self.paths)
    def __getitem__(self,idx):
        img=np.load(self.paths[idx])
#         img=np.vstack(img).transpose((1,0))
       
        if self.transform:
            img=self.transform(image=img)["image"]
        
        label=self.labels[idx]
        return {
            "img":torch.tensor(img,dtype=torch.float32),
            "tar":torch.tensor(label,dtype=torch.float32),
        }

# Model

In [71]:
def get_model():
    model=timm.create_model("tf_efficientnet_b4", pretrained=False)
    model.conv_stem= nn.Conv2d(6, 48, kernel_size=(3, 3), stride=(2, 2), bias=False)
    num_ftrs=model.classifier.in_features
    model.classifier=nn.Linear(num_ftrs,classes)
    return model   

# Engine

In [72]:
class Engine:
    def __init__(self,model,optimizer):
        self.model=model
        self.optimizer=optimizer
        
    def get_accuracy(self,labels,preds):
        total=labels.shape[0]
        if preds.shape[1]==1:
            l=labels
            p=np.uint8(preds>0.5)
        else:
            p=preds.argmax(1).reshape(-1,1)
            l=labels.argmax(1).reshape(-1,1)
        return np.uint8(l==p).sum()/total

    def predict(self,data_loader):
        self.model.eval()
        list_pr=[]
        for data in data_loader:
            inputs=data["img"].to(self.device)
            outputs=self.model(inputs)
            with torch.no_grad():
                outputs=outputs.cpu()
            list_pr.append(outputs.detach().numpy())
        return list_pr

In [73]:
result_path = "../input/seti-model-e4"
pthes = [os.path.join(result_path,s) for s in os.listdir(result_path) if ".bin" in s]
print(pthes)
states = [torch.load(s) for s in pthes]

['../input/seti-model-e4/model_BERT_fold_0 (1).bin', '../input/seti-model-e4/model_BERT_fold_1 (2).bin', '../input/seti-model-e4/model_BERT_fold_2 (1).bin', '../input/seti-model-e4/model_BERT_fold_4 (1).bin', '../input/seti-model-e4/model_BERT_fold_3(1).bin']


In [74]:
def predict(model,dataloader):
    all_preds=[]
    for state in states:
        model.load_state_dict(state)
        model.to(device)
        model.eval()
        
        preds=[]
        
        with torch.no_grad():
            for data in dataloader:
                inputs=data["img"].to(device)
                outputs=model(inputs)
                outputs=outputs.squeeze(-1)
                preds.append(outputs.cpu().detach().numpy())
            preds=np.concatenate(preds)
            all_preds.append(preds)

    return all_preds
    

In [75]:
test=pd.read_csv('../input/seti-breakthrough-listen/sample_submission.csv')
p="../input/seti-breakthrough-listen/test"
test["path"]=test["id"].apply(lambda x:str(p+"/"+x[0]+"/"+x+".npy"))

In [76]:
test_dataset=SETIDataset(df=test,transform=False)
test_loader=torch.utils.data.DataLoader(test_dataset,batch_size=CFG.test_bs,num_workers=4,shuffle=False)
model=get_model()

all_preds=predict(model,test_loader)

In [77]:
findf = pd.DataFrame(all_preds)
findf = findf.T
findf

,0,1,2,3,4
0,-2.306920,-2.550962,-2.640290,-2.983737,-2.712685
1,-2.673671,-2.595874,-1.351478,-2.571787,-2.483634
2,-1.949582,-2.488239,-2.388399,-2.405704,-2.457988
3,-1.829552,-2.338305,-2.187661,-1.641821,-0.677660
4,-2.296652,-2.553222,-2.271683,-2.820920,-2.925473
...,...,...,...,...,...
39990,-2.235611,-2.301241,-1.708478,-1.650978,-3.044610
39991,-1.493408,-2.624357,-1.833807,-1.998475,-2.963630
39992,-2.411376,-2.534481,-2.456504,-1.792950,-3.029288
39993,-1.910276,-2.554531,-1.310786,0.844829,-2.519958


In [78]:
findf=findf.mean(axis=1)
findf

0       -2.638919
1       -2.335289
2       -2.337982
3       -1.735000
4       -2.573590
           ...   
39990   -2.188184
39991   -2.182736
39992   -2.444920
39993   -1.490144
39994    1.843055
Length: 39995, dtype: float64

In [79]:
test["target"]=findf

del test["path"]
test

,id,target
0,000bf832cae9ff1,-2.638919
1,000c74cc71a1140,-2.335289
2,000f5f9851161d3,-2.337982
3,000f7499e95aba6,-1.735000
4,00133ce6ec257f9,-2.573590
...,...,...
39990,fff701540fb8aad,-2.188184
39991,fff9d92a46ea99a,-2.182736
39992,fffa40260ffe43d,-2.444920
39993,fffbc626c9d269f,-1.490144


In [80]:
test.to_csv("submission.csv",index=False)